# Decision tree Classification

## Preperation 

### Importing needed libraries

In [47]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfVectorizer

### Loading Data

In [48]:
data = pd.read_csv("../../data/220502_train_data_preprocessed.csv", sep=';')
data.head()

,id,label,tweet,n_mentions,hashtags,without_puctioation,tweet_lower,tweet_token,clean_token,clean_hashtags,stemmed_tokens,stemmed_hashtags,lemmatized_tokens,lemmatized_hashtags
0,1,0,@user when a father is dysfunctional and is s...,1,['run'],user when a father is dysfunctional and is so...,user when a father is dysfunctional and is so...,"['when', 'father', 'dysfunctional', 'is', 'sel...","['father', 'dysfunctional', 'selfish', 'drags'...",['run'],"['father', 'dysfunct', 'selfish', 'drag', 'kid...",['run'],"['father', 'dysfunctional', 'selfish', 'drag',...",['run']
1,2,0,@user @user thanks for #lyft credit i can't us...,2,"['lyft', 'disapointed', 'getthanked']",user user thanks for lyft credit i cant use ca...,user user thanks for lyft credit i cant use ca...,"['user', 'for', 'credit', 'cant', 'cause', 'do...","['user', 'credit', 'cant', 'cause', 'dont', 'w...","['lyft', 'disapointed', 'getthanked']","['user', 'credit', 'cant', 'caus', 'dont', 'wh...","['lyft', 'disapoint', 'getthank']","['user', 'credit', 'cant', 'cause', 'dont', 'w...","['lyft', 'disapointed', 'getthanked']"
2,3,0,bihday your majesty,0,[],bihday your majesty,bihday your majesty,['your'],[],[],[],[],[],[]
3,4,0,#model i love u take with u all the time in ...,0,['model'],model i love u take with u all the time in u...,model i love u take with u all the time in u...,"['i', 'with', 'u', 'all', 'time', 'urð\x9f\x93...","['u', 'time', 'urð\x9f\x93±', 'ð\x9f\x92¦ð\x9f...",['model'],"['u', 'time', 'urð\x9f\x93±', 'ð\x9f\x92¦ð\x9f...",['model'],"['u', 'time', 'urð\x9f\x93±', 'ð\x9f\x92¦ð\x9f...",['model']
4,5,0,factsguide: society now #motivation,0,['motivation'],factsguide society now motivation,factsguide society now motivation,"['society', 'motivation']","['society', 'motivation']",['motivation'],"['societi', 'motiv']",['motiv'],"['society', 'motivation']",['motivation']


## Try Up-sample Minority Class

Seperate Minortity Class

In [49]:
data_minority = data[data.label == 1]
data_majority = data[data.label == 0]
print("length majority", len(data_majority))
print("length minority", len(data_minority))

length majority 29720
length minority 2242


Upsample minority class

In [50]:
data_minority = resample(data_minority, replace = True, n_samples=29720, random_state=55)

Combine the classes

In [51]:
data_upsampled = pd.concat([data_majority, data_minority])
data_upsampled.label.value_counts()

0    29720
1    29720
Name: label, dtype: int64

In [52]:
data = data_upsampled

## Feature Selection Lemmatized

In [53]:
features = ["n_mentions", "lemmatized_hashtags", "lemmatized_tokens"]
X = data[features]
y = data.label
X.head()

,n_mentions,lemmatized_hashtags,lemmatized_tokens
0,1,['run'],"['father', 'dysfunctional', 'selfish', 'drag',..."
1,2,"['lyft', 'disapointed', 'getthanked']","['user', 'credit', 'cant', 'cause', 'dont', 'w..."
2,0,[],[]
3,0,['model'],"['u', 'time', 'urð\x9f\x93±', 'ð\x9f\x92¦ð\x9f..."
4,0,['motivation'],"['society', 'motivation']"


## Bag of words model

Create the bag of words from the data

In [54]:
import texthero as hero
from texthero import preprocessing
X["bow_tokens"] = (hero.tfidf(X["lemmatized_tokens"], max_features=5000))
X["bow_hashtags"] = (hero.tfidf(X["lemmatized_hashtags"], max_features=5000))
X

/var/folders/0q/8f0zskws27x_14w3qmfndjfw0000gn/T/ipykernel_14199/3688221143.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["bow_tokens"] = (hero.tfidf(X["lemmatized_tokens"], max_features=5000))
/var/folders/0q/8f0zskws27x_14w3qmfndjfw0000gn/T/ipykernel_14199/3688221143.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["bow_hashtags"] = (hero.tfidf(X["lemmatized_hashtags"], max_features=5000))


,n_mentions,lemmatized_hashtags,lemmatized_tokens,bow_tokens,bow_hashtags
0,1,['run'],"['father', 'dysfunctional', 'selfish', 'drag',...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,2,"['lyft', 'disapointed', 'getthanked']","['user', 'credit', 'cant', 'cause', 'dont', 'w...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,0,[],[],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,0,['model'],"['u', 'time', 'urð\x9f\x93±', 'ð\x9f\x92¦ð\x9f...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,0,['motivation'],"['society', 'motivation']","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...
17051,1,"['libtard', 'sjw', 'liberal', 'politics']","['libtard', 'libtard', 'liberal']","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6518,3,['sandniggers'],"['user', 'user', 'use', 'like', '2', 'hm', 'is...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9183,0,"['brexit', 'hatecrime', 'nationalism']","['postbrexit', 'history', 'brexit', 'nationali...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
156,0,"['knicks', 'golfâ']","['hand', 'barry', 'lied', 'game']","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


Reduce the dimension of the vector

In [55]:
X["bow_tokens"] = (hero.tsne(X["bow_tokens"]))
X["bow_hashtags"] = (hero.tsne(X["bow_hashtags"]))
X

/var/folders/0q/8f0zskws27x_14w3qmfndjfw0000gn/T/ipykernel_14199/2594054688.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["bow_tokens"] = (hero.tsne(X["bow_tokens"]))
/var/folders/0q/8f0zskws27x_14w3qmfndjfw0000gn/T/ipykernel_14199/2594054688.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["bow_hashtags"] = (hero.tsne(X["bow_hashtags"]))


,n_mentions,lemmatized_hashtags,lemmatized_tokens,bow_tokens,bow_hashtags
0,1,['run'],"['father', 'dysfunctional', 'selfish', 'drag',...","[-44.27634048461914, 25.174989700317383]","[-7.433860778808594, -15.336506843566895]"
1,2,"['lyft', 'disapointed', 'getthanked']","['user', 'credit', 'cant', 'cause', 'dont', 'w...","[-17.095165252685547, 16.130037307739258]","[-34.860477447509766, -32.4604606628418]"
2,0,[],[],"[-61.81212615966797, -0.12865515053272247]","[-3.994373321533203, 20.591201782226562]"
3,0,['model'],"['u', 'time', 'urð\x9f\x93±', 'ð\x9f\x92¦ð\x9f...","[-19.474811553955078, -60.077354431152344]","[-18.86371421813965, 39.71250915527344]"
4,0,['motivation'],"['society', 'motivation']","[-30.36427879333496, 6.235439777374268]","[20.11435890197754, 57.742855072021484]"
...,...,...,...,...,...
17051,1,"['libtard', 'sjw', 'liberal', 'politics']","['libtard', 'libtard', 'liberal']","[25.654123306274414, -55.03548812866211]","[41.280391693115234, 18.038806915283203]"
6518,3,['sandniggers'],"['user', 'user', 'use', 'like', '2', 'hm', 'is...","[48.54783630371094, -30.92182159423828]","[30.957664489746094, -48.639408111572266]"
9183,0,"['brexit', 'hatecrime', 'nationalism']","['postbrexit', 'history', 'brexit', 'nationali...","[-8.348377227783203, -23.413721084594727]","[35.726680755615234, -9.232100486755371]"
156,0,"['knicks', 'golfâ']","['hand', 'barry', 'lied', 'game']","[45.90692901611328, -0.9228905439376831]","[-23.783769607543945, -47.45092010498047]"


Split vector into columns

In [56]:
X[["bow_tokens_1", "bow_tokens_"]] = pd.DataFrame(X.bow_tokens.tolist(), index= X.index)
X[["bow_hashtags_1", "bow_hashtags_2"]] = pd.DataFrame(X.bow_hashtags.tolist(), index= X.index)
X

/var/folders/0q/8f0zskws27x_14w3qmfndjfw0000gn/T/ipykernel_14199/2517448426.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[["bow_tokens_1", "bow_tokens_"]] = pd.DataFrame(X.bow_tokens.tolist(), index= X.index)
/var/folders/0q/8f0zskws27x_14w3qmfndjfw0000gn/T/ipykernel_14199/2517448426.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[["bow_tokens_1", "bow_tokens_"]] = pd.DataFrame(X.bow_tokens.tolist(), index= X.index)
/var/folders/0q/8f0zskws27x_14w3qmfndjfw0000gn/T/ipykernel_14199/2517448426.py

,n_mentions,lemmatized_hashtags,lemmatized_tokens,bow_tokens,bow_hashtags,bow_tokens_1,bow_tokens_,bow_hashtags_1,bow_hashtags_2
0,1,['run'],"['father', 'dysfunctional', 'selfish', 'drag',...","[-44.27634048461914, 25.174989700317383]","[-7.433860778808594, -15.336506843566895]",-44.276340,25.174990,-7.433861,-15.336507
1,2,"['lyft', 'disapointed', 'getthanked']","['user', 'credit', 'cant', 'cause', 'dont', 'w...","[-17.095165252685547, 16.130037307739258]","[-34.860477447509766, -32.4604606628418]",-17.095165,16.130037,-34.860477,-32.460461
2,0,[],[],"[-61.81212615966797, -0.12865515053272247]","[-3.994373321533203, 20.591201782226562]",-61.812126,-0.128655,-3.994373,20.591202
3,0,['model'],"['u', 'time', 'urð\x9f\x93±', 'ð\x9f\x92¦ð\x9f...","[-19.474811553955078, -60.077354431152344]","[-18.86371421813965, 39.71250915527344]",-19.474812,-60.077354,-18.863714,39.712509
4,0,['motivation'],"['society', 'motivation']","[-30.36427879333496, 6.235439777374268]","[20.11435890197754, 57.742855072021484]",-30.364279,6.235440,20.114359,57.742855
...,...,...,...,...,...,...,...,...,...
17051,1,"['libtard', 'sjw', 'liberal', 'politics']","['libtard', 'libtard', 'liberal']","[25.654123306274414, -55.03548812866211]","[41.280391693115234, 18.038806915283203]",25.654123,-55.035488,41.280392,18.038807
6518,3,['sandniggers'],"['user', 'user', 'use', 'like', '2', 'hm', 'is...","[48.54783630371094, -30.92182159423828]","[30.957664489746094, -48.639408111572266]",48.547836,-30.921822,30.957664,-48.639408
9183,0,"['brexit', 'hatecrime', 'nationalism']","['postbrexit', 'history', 'brexit', 'nationali...","[-8.348377227783203, -23.413721084594727]","[35.726680755615234, -9.232100486755371]",-8.348377,-23.413721,35.726681,-9.232100
156,0,"['knicks', 'golfâ']","['hand', 'barry', 'lied', 'game']","[45.90692901611328, -0.9228905439376831]","[-23.783769607543945, -47.45092010498047]",45.906929,-0.922891,-23.783770,-47.450920


## Splitting data


In [57]:
Data = X[["n_mentions", "bow_tokens_1", "bow_tokens_", "bow_hashtags_1", "bow_hashtags_2"]]
Data

,n_mentions,bow_tokens_1,bow_tokens_,bow_hashtags_1,bow_hashtags_2
0,1,-44.276340,25.174990,-7.433861,-15.336507
1,2,-17.095165,16.130037,-34.860477,-32.460461
2,0,-61.812126,-0.128655,-3.994373,20.591202
3,0,-19.474812,-60.077354,-18.863714,39.712509
4,0,-30.364279,6.235440,20.114359,57.742855
...,...,...,...,...,...
17051,1,25.654123,-55.035488,41.280392,18.038807
6518,3,48.547836,-30.921822,30.957664,-48.639408
9183,0,-8.348377,-23.413721,35.726681,-9.232100
156,0,45.906929,-0.922891,-23.783770,-47.450920


In [58]:
X_train, X_test, y_train, y_test = train_test_split(Data, y, test_size=0.3, random_state=55)

len(X_train)

41608

## Fitting model

In [59]:
results = []
for i in range(1, 50):
    classifier = DecisionTreeClassifier(random_state=55, max_depth=i)

    model = classifier.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    #print("Accuracy:", accuracy_score(y_test, y_pred))
    #print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    results.append([i,accuracy_score(y_test, y_pred)])

print(results)
print(y_test.value_counts()[0]/len(y_test))
#plot_tree(classifier, max_depth=2)

[[1, 0.6685172723194257], [2, 0.735699865410498], [3, 0.7965455361148497], [4, 0.8115746971736204], [5, 0.8237999102736653], [6, 0.8412404665769403], [7, 0.8648497083894123], [8, 0.8816733961417677], [9, 0.8995065051592642], [10, 0.9141431135038134], [11, 0.9215455361148497], [12, 0.9355652759084792], [13, 0.9502018842530283], [14, 0.9582772543741588], [15, 0.9637729923732615], [16, 0.9691004934948407], [17, 0.9718483624943921], [18, 0.9748766262898161], [19, 0.9768393898609242], [20, 0.9777366532077164], [21, 0.9776244952893675], [22, 0.9785778375953342], [23, 0.9781852848811126], [24, 0.9779048900852401], [25, 0.9776805742485419], [26, 0.9789143113503813], [27, 0.9785217586361598], [28, 0.9785778375953342], [29, 0.9787460744728578], [30, 0.9782413638402871], [31, 0.9784656796769852], [32, 0.9785217586361598], [33, 0.9785217586361598], [34, 0.9785217586361598], [35, 0.9785217586361598], [36, 0.9785217586361598], [37, 0.9785217586361598], [38, 0.9785217586361598], [39, 0.97852175863615

[[1, 0.9278339764313276], [2, 0.9278339764313276], [3, 0.9296068411721764], [4, 0.9297111273334029], [5, 0.9311711335905726], [6, 0.9316925643967046], [7, 0.9320054228803838], [8, 0.9322139952028365], [9, 0.931275419751799], [10, 0.9327354260089686], [11, 0.932318281364063], [12, 0.9316925643967046], [13, 0.9298154134946293], [14, 0.9300239858170821], [15, 0.9292939826884973], [16, 0.9285639795599124], [17, 0.9292939826884973], [18, 0.9271039733027427], [19, 0.9248096777557618], [20, 0.9244968192720826], [21, 0.9206382313067056], [22, 0.9203253728230264], [23, 0.9161539263739702], [24, 0.9144853477943476], [25, 0.9147982062780269], [26, 0.911982479924914], [27, 0.9087496089268954], [28, 0.9072896026697257], [29, 0.9055167379288769], [30, 0.9042653039941599], [31, 0.9036395870268016], [32, 0.9040567316717072], [33, 0.9028052977369903], [34, 0.9028052977369903], [35, 0.903222442381896], [36, 0.9017624361247263], [37, 0.900928146834915], [38, 0.900928146834915], [39, 0.9011367191573678], [40, 0.9008238606736886], [41, 0.8987381374491605], [42, 0.8992595682552925], [43, 0.8989467097716133], [44, 0.8973824173532172], [45, 0.8958181249348212], [46, 0.8966524142246324], [47, 0.8961309834185004], [48, 0.8964438419021796], [49, 0.8957138387735948]]

Dataset split: 0.9278339764313276